In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
from itertools import product
from itertools import combinations_with_replacement
import torch
import time
from torch.autograd import Variable
%matplotlib inline

In [3]:
from utils import *
%load_ext autoreload
%autoreload 2

In [4]:
# adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')

best_users = trec.groupby('workerID').count().sort_values('docID', ascending=False)[:150].index
trec = trec[trec['workerID'].isin(best_users)]

r = pd.Series([2,3,2,3], index=[1,2,0,-2])
trec['label_bin'] = trec['label'].map(r)
trec.head(10)

,topicID,workerID,docID,gold,label,label_bin
0,20002,w1,clueweb09-en0000-66-24091,-1,0,2
1,20002,w1,clueweb09-en0001-31-15410,-1,0,2
2,20002,w1,clueweb09-en0000-05-22942,-1,0,2
3,20002,w1,clueweb09-en0000-05-22943,-1,0,2
4,20002,w1,clueweb09-en0006-85-33191,2,2,3
5,20002,w2,clueweb09-en0000-66-24091,-1,1,2
6,20002,w2,clueweb09-en0001-31-15410,-1,1,2
7,20002,w2,clueweb09-en0000-05-22942,-1,0,2
8,20002,w2,clueweb09-en0000-05-22943,-1,0,2
9,20002,w2,clueweb09-en0006-85-33191,2,-2,3


In [5]:
# testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating')
testframe = create_user_task_ids(trec, 'workerID', 'docID', 'label_bin')
testframe.head()

,topicID,workerID,docID,gold,label,label_bin,task_id,uid,bin
0,20002,w1,clueweb09-en0000-66-24091,-1,0,2,0,0,2
1,20002,w1,clueweb09-en0001-31-15410,-1,0,2,1,0,2
2,20002,w1,clueweb09-en0000-05-22942,-1,0,2,2,0,2
3,20002,w1,clueweb09-en0000-05-22943,-1,0,2,3,0,2
4,20002,w1,clueweb09-en0006-85-33191,2,2,3,4,0,3


In [6]:
completed, values, ind = compute_individual_dist(testframe)

In [7]:
start = time.time()
features = np.empty((ind.shape[0], ind.shape[1]**2*ind.shape[0]))
delta_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
score_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
for user_index in range(values.shape[0]):
#     print(user_index)
    #check for full joint distribution or add a prior later
    if np.sum(ind[user_index]==0) > 0:
        continue
    #compute delta matrices with all other users where applicable
    else:
        #create a mask so that other half of tasks can be used later to find score matrix
        mask = np.random.randint(0,2,values.shape).astype(bool)
        delta_matrices, t_m_i_1, cluster_img = compute_deltas(user_index, completed, values, ind, mask, False, 20)
        features[user_index,:] = cluster_img.flatten()
        delta_matrices_all[user_index,:,:,:] = delta_matrices
        score_matrices, t_m_i_2 = compute_deltas(user_index, completed, values, ind, ~mask, True, 20)
        score_matrices_all[user_index,:,:,:] = score_matrices
#         print(np.sum(t_m_i_1), np.sum(t_m_i_2))
#         if len(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))>0:
#             print(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))
#             print(regret(score_matrices, delta_matrices, \
#                      np.logical_and((t_m_i_1==True), (t_m_i_2==True))))
print(time.time()-start)

67.64314293861389


In [8]:
from sklearn.cluster import KMeans

def kmeans_missing(X, n_clusters, max_iter=10):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        if i > 0:
            # initialize KMeans with the previous set of centroids. this is much
            # faster and makes it easier to check convergence (since labels
            # won't be permuted on every iteration), but might be more prone to
            # getting stuck in local minima.
            cls = KMeans(n_clusters, init=prev_centroids)
        else:
            # do multiple random initializations in parallel
            cls = KMeans(n_clusters, n_jobs=-1)

        # perform clustering on the filled-in data
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = centroids[labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

In [9]:
cluster_img = np.zeros_like(delta_matrices_all)
cluster_img = features.reshape(features.shape[0], features.shape[0], delta_matrices_all.shape[2],delta_matrices_all.shape[2])
num_empty = np.sum(np.all(np.isnan(cluster_img), axis=(2,3)), axis=(1))
print(num_empty)
cluster_img_nonnan = cluster_img[np.ix_(num_empty<delta_matrices_all.shape[0],num_empty<delta_matrices_all.shape[0])]
num_empty_cols = np.sum(np.all(np.isnan(cluster_img), axis=(2,3)), axis=(0))
cluster_img_nonnan = cluster_img[np.ix_(np.logical_and(num_empty<delta_matrices_all.shape[0],num_empty_cols<delta_matrices_all.shape[0]),np.logical_and(num_empty<delta_matrices_all.shape[0],num_empty_cols<delta_matrices_all.shape[0]))]
features_nonnan = cluster_img_nonnan.reshape(cluster_img_nonnan.shape[0], cluster_img_nonnan.shape[0]*delta_matrices_all.shape[2]*delta_matrices_all.shape[2])
print(np.logical_and(num_empty<delta_matrices_all.shape[0],num_empty_cols<delta_matrices_all.shape[0]))
neu = np.logical_and(num_empty<delta_matrices_all.shape[0],num_empty_cols<delta_matrices_all.shape[0])
print(features_nonnan.shape)

[136 140 135 141 138 142 145 146 144 146 146 147 143 146 144 143 138 146
 146 147 147 147 147 146  93 120 134 145 145 146 147 139 145 147 129 130
 142 143 143 147 146 146 147 132 147 146 142 141 129 147 146 147 147 147
 147 145 147 147 147 140 147 141 139 147 139 145 146 147 147 144 147 145
 142 142 147 131 146 147 145 143 147 147 147 144 147 147 147 147 147 147
 147 144 141 147 147 145 141 144 147 126 146 147 142 147 147 145 147 143
 147 145 146 140 145 143 140 144 147 147 144 127 145 147 147 147 147 139
 138 122 146 146 139 133 125 138 147 136 147 140 143 139 144 140 147 147
 146 142 146]
[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True False  True False False False False  True
  True  True  True  True  True  True False  True  True False  True  True
  True  True  True False  True  True False  True False  True  True  True
  True False False False False False False  True False False False  True
 False  True  True False  True  True 

In [10]:
print(neu)

[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True False  True False False False False  True
  True  True  True  True  True  True False  True  True False  True  True
  True  True  True False  True  True False  True False  True  True  True
  True False False False False False False  True False False False  True
 False  True  True False  True  True False False False  True False  True
  True  True False  True  True False  True  True False False False  True
 False False False False False False False  True  True False False  True
  True  True False  True False False  True False False  True False  True
 False  True  True  True  True  True  True  True False False  True  True
  True False False False False  True  True  True  True  True  True  True
  True  True False  True False  True  True  True  True  True False False
 False  True False]


In [11]:
labels, centroids, X_hat = kmeans_missing(features_nonnan, 30, max_iter=20)
print(labels)

[15 23 13 26 13 13 25 13 13 13 13 13 13 13  0 13 13 12  1 21 12 13 13 12 12
 13 24  7 12 13 12 13 13  6 13 13 29 17 13 12 12 13 27 13 12 13 12 13 11 13
 13 13 13 12 12 13 12 12  9 12 22 28 13 12  3 13 28 19 28 13  5  0 10 14  2
 13 13 18 20 16  0  4  0  0  0  0  8  0]


/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:893: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [12]:
def calc_cluster_scoring_matrices(labels, delta_matrices_all, non_empty_users):
    delta_matrices_new=np.zeros_like(delta_matrices_all)
    deltas_used = delta_matrices_all[np.ix_(non_empty_users, non_empty_users)]
    for i in np.unique(labels):
        for j in np.unique(labels):
            missing = ~np.isfinite(deltas_used)
            mu = np.nanmean(deltas_used, 0, keepdims=0)
            filled = np.where(missing, mu, deltas_used)
            cluster_i = np.average(deltas_used[labels==i], axis=0)
            if np.sum(np.sum(np.isnan(cluster_i), (1,2)))>0:
                print(i,j)
                missing = ~np.isfinite(deltas_used)
                mu = np.nanmean(deltas_used, 1, keepdims=1)
                X_hat = np.where(missing, mu, X)
            cluster_j = np.average(cluster_i[labels==j], axis=0)
            deltas_used[np.ix_(labels==i,labels==j)] = cluster_j
    score_matrices_new = np.sign(deltas_used)
    print(np.sum(score_matrices_new==1))
    score_matrices_new[score_matrices_new==-1]=0
    return score_matrices_new, deltas_used

In [13]:
score_matrices_new, deltas_used = calc_cluster_scoring_matrices(labels, delta_matrices_all, neu)

19397


In [14]:
deltas_used = delta_matrices_all[np.ix_(neu, neu)]

In [15]:
print(np.sum(score_matrices_new==0))

11579


In [16]:
deltas_used

array([[[[ 0.28173527, -0.2475809 ],
         [-0.2475809 ,  0.21342653]],

        [[-0.07764172,  0.20195912],
         [-0.12968454,  0.00536714]],

        [[-0.11243716,  0.17345516],
         [-0.03977119, -0.02124681]],

        ..., 
        [[-0.20985173,  0.18062582],
         [-0.15906213,  0.18828804]],

        [[ 0.09202212, -0.12124803],
         [ 0.10947042, -0.08024451]],

        [[-0.21397888,  0.18475297],
         [-0.16273345,  0.19195936]]],


       [[[-0.09072961, -0.11473455],
         [ 0.21690911, -0.01144495]],

        [[ 0.12557026, -0.11687461],
         [-0.11687461,  0.10817895]],

        [[-0.04486883, -0.03885788],
         [ 0.07955944,  0.00416727]],

        ..., 
        [[-0.49802149,  0.13715193],
         [ 0.12910764,  0.23176193]],

        [[-0.00697599, -0.35389358],
         [ 0.20846853,  0.15240104]],

        [[-0.50473496,  0.1438654 ],
         [ 0.12802263,  0.23284693]]],


       [[[-0.14292497, -0.04144647],
         [ 0.171779